In [1]:
import torch
from sklearn.datasets import fetch_california_housing
from src.mssp import MSSP

In [2]:
data = fetch_california_housing()
X = torch.tensor(data.data, dtype=torch.double)
y = torch.tensor(data.target, dtype=torch.double)

In [3]:
RANDOM_SEED = 42
if RANDOM_SEED is not None:
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)


In [4]:
i = torch.randperm(len(X))
i_train = i[:int(len(X)*0.8)]
i_valid = i[int(len(X)*0.8):int(len(X)*0.9)]
i_test = i[int(len(X)*0.9):]

In [5]:
X_train, y_train = X[i_train], y[i_train]
X_valid, y_valid = X[i_valid], y[i_valid]
X_test, y_test = X[i_test], y[i_test]

In [6]:
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

torch.Size([16512, 8]) torch.Size([16512])
torch.Size([2064, 8]) torch.Size([2064])
torch.Size([2064, 8]) torch.Size([2064])


In [7]:
model = MSSP(n_best=120, loss_fn="mse", random_seed=RANDOM_SEED, epochs=10, diversity_ratio=0.75, pow_cross=True)

In [8]:
model.fit(X_train, y_train, X_valid, y_valid)
# FIXME Would expect model.fit(X_train, y_train) to act the sasme as model.fit(X_train, y_train, X_train, y_train)
# Best loss: 0.43007519841194153 after training for 9 epochs Xr Xtr
# Best loss: 0.43007519841194153 after training for 9 epochs Xr Xtr
# Best loss: 0.42431482672691345 after training for 9 epochs Xtr
# Best loss: 0.42431482672691345 after training for 9 epochs Xtr

loss (mse): 0.6296 epoch: 0 , time: 0.36s
loss (mse): 0.4852 epoch: 1 , time: 0.88s
loss (mse): 0.4614 epoch: 2 , time: 0.88s
loss (mse): 0.4476 epoch: 3 , time: 0.83s
loss (mse): 0.4414 epoch: 4 , time: 0.81s
loss (mse): 0.4369 epoch: 5 , time: 0.86s
loss (mse): 0.4340 epoch: 6 , time: 0.82s
loss (mse): 0.4295 epoch: 7 , time: 0.84s
loss (mse): 0.4277 epoch: 8 , time: 0.81s
loss (mse): 0.4257 epoch: 9 , time: 0.83s
Best loss: 0.4257083535194397 after training for 9 epochs


In [11]:
y_pred = model.predict(X_test, top_k=16)
mse = ((y_pred - y_test)**2).mean()
mse

tensor(0.3875, dtype=torch.float64)